In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
from io import BytesIO
from PIL import Image
import tensorflow as tf
import requests
import json
import os

In [3]:
pwd

'e:\\projects\\anidex'

In [4]:
MODEL = tf.keras.models.load_model("animal_classification_model.h5")


ValueError: Unknown layer: 'TFOpLambda'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [13]:
model = tf.keras.models.load_model("animal_classification_model.h5")


ValueError: Unknown layer: 'TFOpLambda'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
from anidex.constants import *
from anidex.utils.common import read_yaml, create_directories, save_json

In [8]:
import mlflow
import mlflow.sklearn
# from urllib.parse import urlparse
from sklearn.model_selection import train_test_split



In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        config = self.config
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "data")
        create_directories([Path(training.root_dir)])

        eval_config = EvaluationConfig(
            path_of_model="model.h5",
            training_data=Path(training_data),
            mlflow_uri="https://dagshub.com/Rajarshi12321/Anidex.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
        )
        return eval_config

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:


class Training:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        # Define the path to the main data folder
        data_dir = os.path.join(self.config.training_data, 'animals')

        # Get the list of class names (animal names)
        classes = os.listdir(data_dir)

        # Split data into train, validation, and test sets for each class
        train_data = []
        valid_data = []
        test_data = []

        for class_name in classes:
            class_dir = os.path.join(data_dir, class_name)
            images = os.listdir(class_dir)
            train_images, temp_images = train_test_split(images, train_size=0.70, shuffle=True, random_state=124)
            valid_images, test_images = train_test_split(temp_images, train_size=0.70, shuffle=True, random_state=124)

            train_data.extend([(os.path.join(class_dir, image), class_name) for image in train_images])
            valid_data.extend([(os.path.join(class_dir, image), class_name) for image in valid_images])
            test_data.extend([(os.path.join(class_dir, image), class_name) for image in test_images])

        # Convert data lists to DataFrames
        train_df = pd.DataFrame(train_data, columns=['imgpath', 'labels'])
        valid_df = pd.DataFrame(valid_data, columns=['imgpath', 'labels'])
        test_df = pd.DataFrame(test_data, columns=['imgpath', 'labels'])

        # Reset index for DataFrame consistency
        train_df = train_df.reset_index(drop=True)
        valid_df = valid_df.reset_index(drop=True)
        test_df = test_df.reset_index(drop=True)

        # Print information about splits
        print("----------Train-------------")
        print(train_df.head(5))
        print(train_df.shape)
        print("--------Validation----------")
        print(valid_df.head(5))
        print(valid_df.shape)
        print("----------Test--------------")
        print(test_df.head(5))
        print(test_df.shape)

        # Set up ImageDataGenerator for image augmentation and preprocessing
        generator = ImageDataGenerator(
            preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
            rotation_range=40 if self.config.params_is_augmentation else 0,
            horizontal_flip=True if self.config.params_is_augmentation else False,
            width_shift_range=0.2 if self.config.params_is_augmentation else 0,
            height_shift_range=0.2 if self.config.params_is_augmentation else 0,
            shear_range=0.2 if self.config.params_is_augmentation else 0,
            zoom_range=0.2 if self.config.params_is_augmentation else 0,
            rescale=1./255,
            validation_split=0.20
        )

        # Set up data generators for training, validation, and testing
        self.train_images = generator.flow_from_dataframe(
            dataframe=train_df,
            x_col='imgpath',
            y_col='labels',
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            shuffle=True,
            subset='training'
        )

        self.val_images = generator.flow_from_dataframe(
            dataframe=valid_df,
            x_col='imgpath',
            y_col='labels',
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            shuffle=False,
            subset='validation'
        )

        self.test_generator = generator.flow_from_dataframe(
            dataframe=test_df,
            x_col='imgpath',
            y_col='labels',
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            shuffle=False
        )

        

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="EfficientNet-B")
            else:
                mlflow.keras.log_model(self.model, "model")

NameError: name 'TrainingConfig' is not defined